In [ ]:
from sklearn.linear_model import LinearRegression

# Crear el pipeline
pipeline_linear = Pipeline([
    ('scaler', StandardScaler()),
    ('linear', LinearRegression())
])

# Entrenar directamente sin GridSearchCV ni RandomizedSearchCV
#ya que no aplica
pipeline_linear.fit(X_train, y_train)

# Predecir y evaluar
y_pred_linear = pipeline_linear.predict(X_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))
r2_linear = r2_score(y_test, y_pred_linear)

# Guardar resultados
results['Linear'] = {'RMSE': rmse_linear, 'R²': r2_linear}
print(f"RMSE (Linear): {rmse_linear}")
print(f"R² (Linear): {r2_linear}")


#-----------------------------Curvas de Aprendizaje Lineal-----------------------------
X = data_social.drop('actual_productivity_score', axis=1).select_dtypes(include=np.number)
y = data_social['actual_productivity_score']
linear_model = LinearRegression()

models_for_learning_curve = {
    'Linear Regression': linear_model
}

for name, estimator in models_for_learning_curve.items():
    print(f"Generando curva de aprendizaje para {name}...")

    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 10) # Genera 10 puntos entre 10% y 100% del tamaño del entrenamiento
    )

    # Convertir los scores de MSE negativo a RMSE (Raíz del Error Cuadrático Medio)
    # RMSE = sqrt(-MSE)
    train_scores_rmse = np.sqrt(-train_scores)
    test_scores_rmse = np.sqrt(-test_scores)
    # Calcular la media y desviación estándar de los scores para cada tamaño de entrenamiento
    train_scores_mean = np.mean(train_scores_rmse, axis=1)
    train_scores_std = np.std(train_scores_rmse, axis=1)
    test_scores_mean = np.mean(test_scores_rmse, axis=1)
    test_scores_std = np.std(test_scores_rmse, axis=1)

    # Graficar la curva de aprendizaje
    plt.figure(figsize=(5, 3))
    plt.title(f"Curva de Aprendizaje - {name}")
    plt.xlabel("Tamaño del Conjunto de Entrenamiento")
    plt.ylabel("RMSE") # Usamos RMSE en el eje Y ya que es más interpretable

    # Graficar la curva de entrenamiento con banda de error
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Puntuación Entrenamiento")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Puntuación Validación Cruzada")

    plt.legend(loc="best")
    plt.ylim(bottom=0) # Asegura que el eje Y comience en 0 o más

    plt.show()

